<a href="https://colab.research.google.com/github/dongzihuang/helloworld/blob/master/Dongzi_Copy_of_LAB_30.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##**Context**
The objective of this task is to detect hate speech in tweets. For the sake of simplicity, we say a tweet contains hate speech if it has a racist or sexist sentiment associated with it. So, the task is to classify racist or sexist tweets from other tweets.

Formally, given a training sample of tweets and labels, where label '1' denotes the tweet is racist/sexist and label '0' denotes the tweet is not racist/sexist, your objective is to predict the labels on the test dataset.

USE train.csv and test.csv

1. TRAIN.CSV - data with outputs/labels (Train/Test)
    1. Train
    2. Test / Validation set
2. TEST.CSV - data with just the X (or Test Set)

# Task
Classify tweets as racist/sexist (label '1') or not (label '0') using the provided training data in "train.csv" and predict labels for the test data in "test.csv".

## Load data


Load the `train.csv` and `test.csv` files into pandas DataFrames.


In [11]:
import pandas as pd

train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

print("Train DataFrame head:")
display(train_df.head())

print("\nTest DataFrame head:")
display(test_df.head())

Train DataFrame head:


,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation



Test DataFrame head:


,id,tweet
0,31963,#studiolife #aislife #requires #passion #dedic...
1,31964,@user #white #supremacists want everyone to s...
2,31965,safe ways to heal your #acne!! #altwaystohe...
3,31966,is the hp and the cursed child book up for res...
4,31967,"3rd #bihday to my amazing, hilarious #nephew..."


## Explore data

Perform exploratory data analysis to understand the structure and content of the datasets, including the distribution of labels in the training data.



Perform exploratory data analysis by displaying the shape, data types, missing values, label distribution, and sample tweets for both the training and testing dataframes as instructed.



In [12]:
# 1. Display the shape of both dataframes
print("Shape of train_df:", train_df.shape)
print("Shape of test_df:", test_df.shape)

# 2. Print the data types of the columns
print("\nData types of train_df columns:")
display(train_df.info())
print("\nData types of test_df columns:")
display(test_df.info())

# 3. Check for missing values
print("\nMissing values in train_df:")
display(train_df.isnull().sum())
print("\nMissing values in test_df:")
display(test_df.isnull().sum())

# 4. Analyze the distribution of the 'label' column in train_df
print("\nDistribution of 'label' in train_df:")
label_counts = train_df['label'].value_counts()
label_percentages = train_df['label'].value_counts(normalize=True) * 100
display(pd.DataFrame({'counts': label_counts, 'percentages': label_percentages}))

# 5. Display a few sample tweets from both labels in train_df
print("\nSample tweets with label 0:")
display(train_df[train_df['label'] == 0]['tweet'].sample(5).tolist())

print("\nSample tweets with label 1:")
display(train_df[train_df['label'] == 1]['tweet'].sample(5).tolist())

Shape of train_df: (31962, 3)
Shape of test_df: (17197, 2)

Data types of train_df columns:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31962 entries, 0 to 31961
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      31962 non-null  int64 
 1   label   31962 non-null  int64 
 2   tweet   31962 non-null  object
dtypes: int64(2), object(1)
memory usage: 749.2+ KB


None


Data types of test_df columns:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17197 entries, 0 to 17196
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      17197 non-null  int64 
 1   tweet   17197 non-null  object
dtypes: int64(1), object(1)
memory usage: 268.8+ KB


None


Missing values in train_df:


...

,counts,percentages
label,,
0,29720,92.98542
1,2242,7.01458



Sample tweets with label 0:


["@user @user it's an a &amp; takes a ceain kind of person to scam &amp; be scammed. vulnerable &amp; lonely ppl  given some attn. ",
 'johnny depp is no longer remotely hot and  queer dance clubs are mass killing grounds  #tired   #wearyasfuck #dontwanttopray',
 'on the right track!  ',
 '#aymtracklist no seasoned aist vocals #kahik #chimayi #shakthishreegopalan   ',
 'severely disappointed with #mtscentre closing all the beer sales off early when hosting #meatloaf  really, 9 pm for a conce?   #wpg']


Sample tweets with label 1:


['the end of   #me #selfie # #love #messi #cr7 #religion #christianity #mecca_live ',
 '@user ppl wait for 10 hrs @user airpo, miss connecting flights, put them on flights 24 hrs later, aâ\x80¦ ',
 '@user what2inlieuof #narcissimâ\x80\x93#misogynyâ\x80\x93? #tolerance #kindness #peace stop w/#hate. look inside you are #love!â\x80¦ ',
 'critics threaten boycotts of simon &amp; schuster over milo yiannopoulos book deal | the huffington post  ',
 'more evidence of the impoance of  in the #brexit vote ']

## Preprocess Text Data


Clean and preprocess the tweet text data. This may include steps like lowercasing, removing punctuation, stop words, and potentially stemming or lemmatization.


Clean the tweet text by removing special characters, URLs, mentions, and hashtags. Convert the text to lowercase, remove stop words, and apply stemming or lemmatization to reduce words to their root form. This will help in creating better features for the model.

In [13]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer

# Download necessary NLTK data
nltk.download('stopwords')
nltk.download('wordnet')

stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()

def preprocess_tweet(tweet):
    # Remove special characters and URLs
    tweet = re.sub(r"http\S+|www\S+|https\S+", '', tweet, flags=re.MULTILINE)
    tweet = re.sub(r'\@\w+|\#','', tweet) # Remove mentions and hashtags
    tweet = re.sub(r'[^\w\s]', '', tweet) # Remove punctuation

    # Convert to lowercase
    tweet = tweet.lower()

    # Remove stop words and perform stemming or lemmatization
    # Using lemmatization as it often performs better than stemming
    tweet_tokens = tweet.split()
    cleaned_tweet = [lemmatizer.lemmatize(word) for word in tweet_tokens if word not in stop_words]

    return " ".join(cleaned_tweet)

# Apply preprocessing to both train and test data
train_df['cleaned_tweet'] = train_df['tweet'].apply(preprocess_tweet)
test_df['cleaned_tweet'] = test_df['tweet'].apply(preprocess_tweet)

print("Train DataFrame with cleaned tweets:")
display(train_df[['tweet', 'cleaned_tweet']].head())

print("\nTest DataFrame with cleaned tweets:")
display(test_df[['tweet', 'cleaned_tweet']].head())

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Train DataFrame with cleaned tweets:


,tweet,cleaned_tweet
0,@user when a father is dysfunctional and is s...,father dysfunctional selfish drag kid dysfunct...
1,@user @user thanks for #lyft credit i can't us...,thanks lyft credit cant use cause dont offer w...
2,bihday your majesty,bihday majesty
3,#model i love u take with u all the time in ...,model love u take u time urð ðððð ððð
4,factsguide: society now #motivation,factsguide society motivation



Test DataFrame with cleaned tweets:


,tweet,cleaned_tweet
0,#studiolife #aislife #requires #passion #dedic...,studiolife aislife requires passion dedication...
1,@user #white #supremacists want everyone to s...,white supremacist want everyone see new â bird...
2,safe ways to heal your #acne!! #altwaystohe...,safe way heal acne altwaystoheal healthy healing
3,is the hp and the cursed child book up for res...,hp cursed child book reservation already yes ð...
4,"3rd #bihday to my amazing, hilarious #nephew...",3rd bihday amazing hilarious nephew eli ahmir ...


In [14]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [22]:
from sklearn.feature_extraction.text import CountVectorizer

# Initialize Count Vectorizer
# You can adjust parameters like max_features, min_df, max_df, ngram_range
# These parameters help you to fine-tune the feature engineering process by controlling the size and content of the vocabulary created from the text data.
count_vectorizer = CountVectorizer(max_features=5000, min_df=5, max_df=0.9)

# Fit and transform the training data, and transform the test data
X_train = count_vectorizer.fit_transform(train_df['cleaned_tweet'])
X_test = count_vectorizer.transform(test_df['cleaned_tweet'])

# Get the labels for the training data
y_train = train_df['label']

print("Shape of X_train:", X_train.shape)
print("Shape of X_test:", X_test.shape)
print("Shape of y_train:", y_train.shape)

count_vectorizer: CountVectorizer(max_df=0.9, max_features=5000, min_df=5)
Shape of X_train: (31962, 5000)
Shape of X_test: (17197, 5000)
Shape of y_train: (31962,)


## Train Model


Train a classification model on the preprocessed training data.


Train a Logistic Regression model on the training data (`X_train`, `y_train`). Logistic Regression is a good starting point for binary classification tasks and is computationally efficient.

In [16]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Split the training data into training and validation sets
# This is a good practice to evaluate the model's performance on unseen data during training
X_train_split, X_val, y_train_split, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42, stratify=y_train)

# Initialize and train the Logistic Regression model
model = LogisticRegression(random_state=42, solver='liblinear') # Using liblinear solver for smaller datasets
model.fit(X_train_split, y_train_split)

print("Model training complete.")

Model training complete.


## Evaluate Model

Evaluate the trained model's performance on the validation set.


Evaluate the Logistic Regression model on the validation set (`X_val`, `y_val`) using accuracy, precision, recall, and F1-score to assess its performance in classifying hate speech.

In [17]:
# Predict on the validation set
y_pred = model.predict(X_val)

# Evaluate the model
accuracy = accuracy_score(y_val, y_pred)
precision = precision_score(y_val, y_pred)
recall = recall_score(y_val, y_pred)
f1 = f1_score(y_val, y_pred)

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")

Accuracy: 0.9570
Precision: 0.8168
Recall: 0.4978
F1-score: 0.6186


## Predict on Test Data

Use the trained model to predict labels on the unseen test dataset.


Use the trained Logistic Regression model to predict the labels for the `X_test` data.

In [18]:
# Predict on the test data
test_predictions = model.predict(X_test)

# Add predictions to the test_df
test_df['label'] = test_predictions

print("Test DataFrame with predictions:")
display(test_df.head())

Test DataFrame with predictions:


,id,tweet,cleaned_tweet,label
0,31963,#studiolife #aislife #requires #passion #dedic...,studiolife aislife requires passion dedication...,0
1,31964,@user #white #supremacists want everyone to s...,white supremacist want everyone see new â bird...,0
2,31965,safe ways to heal your #acne!! #altwaystohe...,safe way heal acne altwaystoheal healthy healing,0
3,31966,is the hp and the cursed child book up for res...,hp cursed child book reservation already yes ð...,0
4,31967,"3rd #bihday to my amazing, hilarious #nephew...",3rd bihday amazing hilarious nephew eli ahmir ...,0


In [ ]:
# Create a submission DataFrame
submission_df = test_df[['id', 'label']]

# Save the submission file
submission_df.to_csv('submission.csv', index=False)

print("Submission file created successfully!")
display(submission_df.head())

Submission file created successfully!


,id,label
0,31963,0
1,31964,0
2,31965,0
3,31966,0
4,31967,0


## Finish task

Summarize the approach and present the results.


Provide a summary of the steps taken to address the problem, including data loading, preprocessing, feature engineering, model training, and evaluation. Present the evaluation metrics obtained on the validation set and mention the creation of the submission file.

## **Summary**:
The task was to classify tweets as racist/sexist (label '1') or not (label '0').
1. Data was loaded from `train.csv` and `test.csv`.
2. Exploratory data analysis was performed to understand the data distribution. The training data showed an imbalance in the class distribution.
3. Tweet text data was preprocessed by removing special characters, URLs, mentions, hashtags, converting to lowercase, and applying lemmatization.
4. Count Vectorization was used to convert the cleaned text into numerical features.
5. A Logistic Regression model was trained on the preprocessed training data.
6. The model was evaluated on a validation set, achieving the following metrics:
    - Accuracy: 0.9570
    - Precision: 0.8168
    - Recall: 0.4978
    - F1-score: 0.6186
7. Predictions were made on the test dataset, and the results were saved to `submission.csv` with 'id' and 'label' columns.
